### IMPORT LIBRARIES

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import os 
import tqdm
import glob
import tensorflow 

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.color import grey2rgb

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, BatchNormalization, Dropout, Flatten, Dense, Activation, MaxPool2D, Conv2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.densenet import DenseNet169
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import backend as K
from tensorflow.keras.layers import MaxPooling2D

ModuleNotFoundError: No module named 'tensorflow'

### IMPORT / VIEWING / PREPROCESSING DATASET

> `DATA AUGMENTATION`

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=30,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = 0.2)

valid_datagen = ImageDataGenerator(rescale = 1./255,
                                   validation_split = 0.2)

test_datagen  = ImageDataGenerator(rescale = 1./255)

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
train_dataset  = train_datagen.flow_from_directory(directory = './Alzheimer_s Dataset/train',
                                                   target_size = (150,150),
                                                   class_mode = 'categorical',
                                                   subset = 'training',
                                                   batch_size = 128)

In [ ]:
valid_dataset = valid_datagen.flow_from_directory(directory = './Alzheimer_s Dataset/train',
                                                  target_size = (150,150),
                                                  class_mode = 'categorical',
                                                  subset = 'validation',
                                                  batch_size = 128)

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 5, figsize=(20,20))

for i in tqdm(range(0,5)):
    rand1 = np.random.randint(len(train_dataset))
    rand2 = np.random.randint(100)
    ax[i].imshow(train_dataset[rand1][0][rand2])
    ax[i].axis('off')
    a = train_dataset[rand1][1][rand2]
    if a[0] == 1:
        ax[i].set_title('Mild Dementia')
    elif a[1] == 1:
        ax[i].set_title('Moderate Dementia')
    elif a[2] == 1:
        ax[i].set_title('Non Demetia')
    elif a[3] == 1:
        ax[i].set_title('Very Mild Dementia')

### MODEL BUILDING

In [ ]:
EPOCHS = 10
STEPS = 100
LR = 1e-3
BATCH_SIZE = 10
WIDTH = 150
HEIGHT = 150
DEPTH = 3

In [4]:

model = Sequential()
inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1



if K.image_data_format() == "channels_first":
    inputShape = (DEPTH, HEIGHT, WIDTH)
    chanDim = 1

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation("softmax"))

model.summary()

NameError: name 'Sequential' is not defined

In [5]:
# Model Compile 

OPT    = tensorflow.keras.optimizers.Adam(lr=0.001)

model.compile(loss='categorical_crossentropy',
              metrics=[tensorflow.keras.metrics.AUC(name = 'auc')],
              optimizer=OPT)

NameError: name 'tensorflow' is not defined

In [9]:
# Defining Callbacks

filepath = './alz.hdf5'

earlystopping = EarlyStopping(monitor = 'val_auc', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_auc', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
model_history=model.fit(train_dataset,
                        validation_data=valid_dataset,
                        epochs = 10,
                        callbacks = callback_list,
                        verbose = 1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/10
32/33 [============================>.] - ETA: 6s - loss: 1.9507 - auc: 0.6318 
Epoch 00001: val_auc improved from -inf to 0.79595, saving model to ./alz.hdf5
33/33 [==============================] - 235s 7s/step - loss: 1.9267 - auc: 0.6354 - val_loss: 1.6509 - val_auc: 0.7959
Epoch 2/10
32/33 [============================>.] - ETA: 6s - loss: 1.3004 - auc: 0.7407 
Epoch 00002: val_auc did not improve from 0.79595
33/33 [==============================] - 215s 7s/step - loss: 1.2964 - auc: 0.7421 - val_loss: 4.2907 - val_auc: 0.7342
Epoch 3/10
32/33 [============================>.] - ETA: 6s - loss: 1.2415 - auc: 0.7800 
Epoch 00003: val_auc did not improve from 0.79595
33/33 [==============================] - 211s 6s/step - loss: 1.2352 - auc: 0.7806 - val_loss: 5.4651 - val_auc: 0.6871
Epoch 4/10
32/33 [============================>.] - ETA: 6s - loss: 1.1767 - auc: 0.8011 
Epoch 0000

### MODEL EVALUATION

In [ ]:
# Summarize history for loss

plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()

In [2]:
# Summarize history for loss

plt.plot(model_history.history['auc'])
plt.plot(model_history.history['val_auc'])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Test Data 

test_dataset  = test_datagen.flow_from_directory(directory = './Alzheimer_s Dataset/test',
                                                 target_size = (150,150),
                                                 class_mode = 'categorical',
                                                 batch_size = 128)

In [ ]:
# Evaluating Loss and AUC

model.evaluate(test_dataset)

In [ ]:
dic = test_dataset.class_indices
idc = {k:v for v, k in dic.items()}

img = load_img('./Alzheimer_s Dataset/test/NORMAL/26 (62).jpg', target_size = (150,150,3))
img = img_to_array(img)
img = img/255
imshow(img)
plt.axis('off')
img = np.expand_dims(img,axis=0)
#answer = model.predict_classes(img)
answer = np.argmax(model.predict(img))

print( ' chances are that the image is',idc[answer])

In [1]:
dic = test_dataset.class_indices
idc = {k:v for v, k in dic.items()}

img = load_img('./Alzheimer_s Dataset/test/VeryMild/26 (47).jpg', target_size = (150,150,3))
img = img_to_array(img)
img = img/255
imshow(img)
plt.axis('off')
img = np.expand_dims(img,axis=0)
#answer = model.predict_classes(img)
answer = np.argmax(model.predict(img))

print( ' chances are that the image is',idc[answer])

NameError: name 'test_dataset' is not defined